Loading Data:

In [382]:
import pandas as pd

# Load the Excel file into a DataFrame
data = pd.read_excel("Survey_ICS487_Project.xlsx")


Data Inspection:

In [383]:
# Check the first few rows
print(data.head())

# Get an overview of the data
print(data.info())

# Check for missing values
print(data.isnull().sum())


     Name  Salary (SAR)  Age    State     Sex  Monthly Debt (SAR)  \
0  faisal          2447   27  Married  Female              509.27   
1    Yara          3093   18   Single  Female              826.77   
2   bader           978   49  Married    Male              222.84   
3   Rawan          1187   59   Single    Male              255.64   
4   turki         13014   25   Single  Female             3536.82   

   Elementary Expenses (SAR)     Goal  Number of Children Employment_Status  \
0                   14121.45  Savings                   1           Student   
1                    3174.57  Savings                   0           Student   
2                    6393.58  Savings                   3        Unemployed   
3                    3451.12  Savings                   0        Unemployed   
4                     667.08  Savings                   0          Employed   

  Nationality Budgeting_Rule Financial_Comfortability Goal_Progress  
0       Saudi       50/30/20            

Clean and Preprocess:

In [384]:
# Standardize column names
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]
# Check for duplicates and remove them
data = data.drop_duplicates()

# Handle missing values (drop rows or fill them as needed)
data = data.dropna()  # Drop rows with missing values
# Alternatively, fill missing values (example: fill salary with median)
data['salary_(sar)'] = data['salary_(sar)'].fillna(data['salary_(sar)'].median())

# Ensure numeric columns are properly formatted
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Validate ranges for numeric columns
data = data[(data['age'] >= 18) & (data['age'] <= 100)]  # Age should be between 0 and 100
data = data[data['salary_(sar)'] >= 0]  # Salary should not be negative

# Filter out rows where the conditions are met
filtered_data = data[
    (data['nationality'].str.lower() == 'saudi') ]  # Keep only rows where nationality is 'saudi'
filtered_data = filtered_data[
    (filtered_data['financial_comfortability'].str.lower() != 'no') ]  # Exclude rows where financial_comfortability is 'no'
filtered_data = filtered_data[
    (data['goal_progress'].str.lower() != 'no')  # Exclude rows where goal_progress is 'no'
]

# Drop the irrelevant columns
filtered_data = filtered_data.drop(columns=['nationality', 'financial_comfortability', 'goal_progress'])

# Store the filtered data back if needed
data = filtered_data
print(data.tail())



          name  salary_(sar)  age    state     sex  monthly_debt_(sar)  \
1170     tariq         28942   36  Married  Female             2034.19   
1171  Khadijah         15446   24   Single    Male             1457.07   
1173     osamh          3011   20   Single  Female              617.84   
1174  Khadijah         19058   20   Single    Male             5275.05   
1175    sultan         26038   58  Married    Male             6646.54   

      elementary_expenses_(sar)        goal  number_of_children  \
1170                    6530.04  Investment                   1   
1171                     511.71  Investment                   0   
1173                    1427.96  Investment                   0   
1174                    3602.68     Savings                   0   
1175                   10331.68  Investment                   5   

     employment_status budgeting_rule  
1170          Employed       50/30/20  
1171          Employed       50/30/20  
1173           Student       60/

/var/folders/yc/7vmb_7z55z3d92njbdlxxr3m0000gp/T/ipykernel_61078/1195411881.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[


Removing outliers:

In [385]:
import numpy as np

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to numeric columns with potential outliers
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


# Define age bins and labels
age_bins = [0, 24, 34, 44, 54, float('inf')]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']

# Create a new column for age groups
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True)

data = data.drop(columns=['age'])


Encoding Categorical Variables:

In [386]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical columns
categorical_columns = ['state', 'sex', 'goal', 'employment_status']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Encode the age_group using LabelEncoder
label_encoder = LabelEncoder()
data['age_group_encoded'] = label_encoder.fit_transform(data['age_group'])

# Display the encoding
print("Age Group Encoding:")
for label, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{label}: {encoded}")

data = data.drop(columns=['age_group'])


Age Group Encoding:
18-24: 0
25-34: 1
35-44: 2
45-54: 3
55+: 4


Feature Engineering:

In [387]:
# Create new features
data['debt_to_income_ratio'] = data['monthly_debt_(sar)'] / data['salary_(sar)']
data['savings_ratio'] = data['elementary_expenses_(sar)'] / data['salary_(sar)']

# Handle infinite or NaN values due to division
data['debt_to_income_ratio'] = data['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)
data['savings_ratio'] = data['savings_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop irrelevant columns
data = data.drop(columns=['name'])

data.describe()

,salary_(sar),monthly_debt_(sar),elementary_expenses_(sar),number_of_children,age_group_encoded,debt_to_income_ratio,savings_ratio
count,745.000000,745.000000,745.000000,745.000000,745.000000,745.000000,745.000000
mean,4051.404027,506.025584,5525.033691,1.194631,1.326174,0.151673,2.274469
std,3829.980334,399.872256,3555.717335,1.714415,1.281454,0.082497,2.247365
min,904.000000,50.100000,467.560000,0.000000,0.000000,0.003592,0.033129
25%,1791.000000,213.470000,2827.240000,0.000000,0.000000,0.078473,0.761463
50%,2733.000000,388.860000,4663.130000,0.000000,1.000000,0.149955,1.597965
75%,4427.000000,679.020000,7947.840000,2.000000,2.000000,0.225090,3.007356
max,20214.000000,1887.210000,14839.350000,5.000000,4.000000,0.299707,15.361976


Saving Data:

In [388]:
# Save cleaned data to a new Excel file
data.to_excel("Cleaned_Survey_ICS487_Project.xlsx", index=False)

# Or save it as a CSV file
data.to_csv("Cleaned_Survey_ICS487_Project.csv", index=False)